In [1]:
from alignment import *
from read_data import *
import pulp as plp
import numpy as np
import pandas as pd

In [2]:
d=node('d',[])
b=node('b',[d])
c=node('c',[])
a=node('a',[b,c]);

In [3]:
f=node('f',[])
g=node('g',[])
e=node('e',[f,g])

In [2]:
folder_path1 = './datas/118/1/'
folder_path2 = './datas/118/2/'


In [3]:
nodes1,n1 = build_hyper_tree(folder_path1)
nodes2,n2 = build_hyper_tree(folder_path2)
root1 = nodes1[0]
root2 = nodes2[0]
meta_list1 = np.load(folder_path1+'tree_merge.npy')
meta_list2 = np.load(folder_path2+'tree_merge.npy')

In [4]:
for i in range(len(nodes1)):
    if(int(nodes1[i].name)<len(meta_list1)):
        nodes1[i].name= nodes1[i].name +'_'+ meta_list1[int(nodes1[i].name)];
for i in range(len(nodes2)):
    if(int(nodes2[i].name)<len(meta_list2)):
        nodes2[i].name= nodes2[i].name +'_'+ meta_list2[int(nodes2[i].name)];  

In [5]:
values1 = np.array([i.value for i in nodes1])
values2 = np.array([i.value for i in nodes2])
similarities =np.zeros((len(values1),len(values2)))
for i in range(len(values1)):
    for j in range(len(values2)):
        similarities[i][j]=np.corrcoef(values1[i],values2[j])[0][1]
similarities

array([[0.74928719, 0.73958779, 0.71384371, 0.86654661, 0.93088676,
        0.56373045, 0.32537296, 0.27830668, 0.21226157, 0.45681294,
        0.47684114, 0.43071632, 0.43156085, 0.4289529 , 0.49717019,
        0.41349937, 0.55118406],
       [0.71644909, 0.73740511, 0.71165538, 0.86465027, 0.92626563,
        0.56613812, 0.25208034, 0.20880031, 0.14922985, 0.39476333,
        0.41472162, 0.36909326, 0.36753559, 0.36880214, 0.43814065,
        0.36122913, 0.48950383],
       [0.69577245, 0.71326327, 0.68768929, 0.84174612, 0.9240559 ,
        0.5196789 , 0.25039106, 0.20745948, 0.14755261, 0.39596216,
        0.41573646, 0.37050189, 0.36862249, 0.37037853, 0.43475058,
        0.35882888, 0.48524858],
       [0.73279904, 0.78904795, 0.76961633, 0.8594367 , 0.64893925,
        0.94566118, 0.18984799, 0.1565417 , 0.12063103, 0.25051654,
        0.26615914, 0.230724  , 0.2335938 , 0.22848664, 0.3354996 ,
        0.27180139, 0.38052342],
       [0.79110292, 0.83824256, 0.81414804, 0.940969

In [6]:
def find_path_root(now,dfs,path,dfs_node,f):
    now.path=path.copy();
    now.f=f
    now.dfs=dfs;
    path.append(now);
    dfs_node.append(now);
    for i in now.son:
        dfs=find_path_root(i,dfs+1,path,dfs_node,now);
        
    path.remove(now)
    now.num_son = dfs-now.dfs;
    return dfs
n = n1
m = n2
set_I = range(0, n)
set_J = range(0, m)
c = {(i,j): similarities[i][j] for i in set_I for j in set_J}


In [7]:
dfs_node1=[]
dfs_node2=[]
find_path_root(root1,0,[],dfs_node1,root1)
find_path_root(root2,0,[],dfs_node2,root2)

16

In [8]:
x_vars  = {(i,j):plp.LpVariable(cat=plp.LpBinary, name="x_{0}_{1}".format(i,j)) for i in set_I for j in set_J}


In [9]:
opt_model = plp.LpProblem(name="MIP Model")
for i in set_I:
    opt_model.addConstraint(
    plp.LpConstraint(e=plp.lpSum(x_vars[i,j] for j in set_J),
                     sense=plp.LpConstraintGE,
                     rhs=1,
                     name="constraintI{0}".format(i)))

for j in set_J:
    opt_model.addConstraint(
    plp.LpConstraint(e=plp.lpSum(x_vars[i,j] for i in set_I),
                     sense=plp.LpConstraintGE,
                     rhs=1,
                     name="constraintJ{0}".format(j))) 
for i in dfs_node1:
    for j in dfs_node2:
        son_i = [dfs_node1[k] for k in range(i.dfs,i.dfs+i.num_son)];
        son_j = [dfs_node2[k] for k in range(j.dfs,j.dfs+j.num_son)];
        for k in i.path:
            for l in j.path:
                if(k==[]or l==[]):
                    continue;
                # print(i,j,k,l)
                opt_model.addConstraint(
                plp.LpConstraint(e=x_vars[i.dfs,j.dfs]+x_vars[i.dfs,l.dfs]+x_vars[k.dfs,j.dfs],
                                sense=plp.LpConstraintLE,
                                rhs=2,
                                name="constraint{}_{}_{}_{}_1".format(i,j,k,l)))
for i in dfs_node1:
    if(len(i.son)==2):
        l=i.son[0];
        r=i.son[1];
        for j in dfs_node2:
            opt_model.addConstraint(
            plp.LpConstraint(e=x_vars[l.dfs,j.dfs]+x_vars[r.dfs,j.dfs]-x_vars[i.dfs,j.dfs],
                                sense=plp.LpConstraintLE,
                                rhs=1,
                                name="constraint{}_{}_{}_{}_1".format(i,j,k,l)))
                

objective = plp.lpSum(x_vars[i,j] * c[i,j] for i in set_I for j in set_J)
opt_model.sense = plp.LpMaximize
opt_model.setObjective(objective)
opt_model.solve()

/Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/bm/69jvgsdj3rxch558kx9yp9y00000gn/T/1d8da34d978f496ebfa7d1cca9cf8ccc-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/bm/69jvgsdj3rxch558kx9yp9y00000gn/T/1d8da34d978f496ebfa7d1cca9cf8ccc-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1425 COLUMNS
At line 6701 RHS
At line 8122 BOUNDS
At line 8344 ENDATA
Problem MODEL has 1420 rows, 221 columns and 4612 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 77.3384 - 0.03 seconds
Cgl0004I processed model has 1417 rows, 220 columns (220 integer (220 of which binary)) and 4579 elements
Cbc0038I Initial state - 184 integers unsatisfied sum - 61.3333
Cbc0038I Pass   1: suminf.   50.66667 (152) obj. -71.6195 iterations 395

1

In [10]:
print('SOLUTION:')
for v in opt_model.variables():
    print(f'\t\t{v.name} = {v.varValue}')

print('\n') # Prints a blank line
print(f'OBJECTIVE VALUE: {opt_model.objective.value()}')

SOLUTION:
		x_0_0 = 1.0
		x_0_1 = 1.0
		x_0_10 = 1.0
		x_0_11 = 1.0
		x_0_12 = 1.0
		x_0_13 = 1.0
		x_0_14 = 1.0
		x_0_15 = 1.0
		x_0_16 = 1.0
		x_0_2 = 1.0
		x_0_3 = 1.0
		x_0_4 = 1.0
		x_0_5 = 1.0
		x_0_6 = 1.0
		x_0_7 = 1.0
		x_0_8 = 1.0
		x_0_9 = 1.0
		x_10_0 = 0.0
		x_10_1 = 0.0
		x_10_10 = 1.0
		x_10_11 = 0.0
		x_10_12 = 1.0
		x_10_13 = 1.0
		x_10_14 = 0.0
		x_10_15 = 1.0
		x_10_16 = 1.0
		x_10_2 = 1.0
		x_10_3 = 0.0
		x_10_4 = 1.0
		x_10_5 = 1.0
		x_10_6 = 0.0
		x_10_7 = 0.0
		x_10_8 = 1.0
		x_10_9 = 0.0
		x_11_0 = 0.0
		x_11_1 = 0.0
		x_11_10 = 1.0
		x_11_11 = 0.0
		x_11_12 = 1.0
		x_11_13 = 1.0
		x_11_14 = 0.0
		x_11_15 = 1.0
		x_11_16 = 1.0
		x_11_2 = 1.0
		x_11_3 = 0.0
		x_11_4 = 1.0
		x_11_5 = 1.0
		x_11_6 = 0.0
		x_11_7 = 0.0
		x_11_8 = 1.0
		x_11_9 = 0.0
		x_12_0 = 0.0
		x_12_1 = 0.0
		x_12_10 = 1.0
		x_12_11 = 0.0
		x_12_12 = 1.0
		x_12_13 = 1.0
		x_12_14 = 0.0
		x_12_15 = 1.0
		x_12_16 = 1.0
		x_12_2 = 1.0
		x_12_3 = 0.0
		x_12_4 = 1.0
		x_12_5 = 1.0
		x_12_6 = 0.0
		x_

In [11]:
result_node = []
for v in opt_model.variables():
    if(v.value()==0):
        continue
    l = int(v.name.split('_')[1])
    r = int(v.name.split('_')[2])
    tn = newnode(dfs_node1[l],dfs_node2[r])
    result_node.append(tn);
    
for i in range(len(result_node)):
    for j in range(len(result_node)):
        if(i==j):
            continue
        p1 = result_node[i]
        p2 = result_node[j]
        if((p1.node1 in p2.node1.path or p1.node1 == p2.node1) and (p1.node2 in p2.node2.path or p1.node2==p2.node2)):
            result_node[i].edge.append(result_node[j])
            result_node[j].indegree +=1

def find_indgee(lists,indegree):
    ans=[]
    for i in lists:
        if(i.indegree == indegree):
            ans.append(i);
    return ans;
root=find_indgee(result_node,0)[0]
root

12_16

In [12]:
type(root)

alignment.newnode

In [14]:
c=0;z=0

for i in result_node:
    ans = str(i).split('_')
    if(len(ans) == 4):
        c+=1
        if(ans[1] == ans[3]):
            z+=1
print('correct alignment rate:{}'.format(z/c))


correct alignment rate:0.3333333333333333


In [11]:
opt_model.variables()[0].varValue

1.0

In [5]:
c0=node('b7',[])
d0=node('b8',[])
f0=node('b3',[])
h0=node('b5',[])
b0=node('b6',[c0,d0])
g0=node('b4',[h0])
e0=node('b2',[f0,g0])
a0=node('b1',[b0,e0])



In [6]:
f1=node('r9',[])
g1=node('r10',[])
d1=node('r7',[])
e1=node('r8',[f1,g1])
c1=node('r6',[d1,e1])
b1=node('r5',[c1])
j1=node('r4',[])
i1=node('r3',[j1])
h1=node('r2',[i1])
a1=node('r1',[b1,h1])

In [18]:
def find_path_root(now,dfs,path,dfs_node,f):
    now.path=path.copy();
    now.f=f
    now.dfs=dfs;
    path.append(now);
    dfs_node.append(now);
    for i in now.son:
        dfs=find_path_root(i,dfs+1,path,dfs_node,now);
        
    path.remove(now)
    now.num_son = dfs-now.dfs;
    return dfs
n = 8
m = 10
set_I = range(0, n)
set_J = range(0, m)
c = {(i,j): 0 for i in set_I for j in set_J}


In [24]:
temp = [(1,1),(5,8),(6,8),(7,9),(8,9),(7,10),(1,2),(1,3),(2,2),(3,2),(4,2),(1,4),(1,5),(1,6),(1,7)]
for i,j in temp:
    c[(i-1,j-1)]=10;

In [25]:
dfs_node1=[]
dfs_node2=[]
find_path_root(a0,0,[],dfs_node1,a0)
find_path_root(a1,0,[],dfs_node2,a1)
x_vars  = {(i,j):plp.LpVariable(cat=plp.LpBinary, name="x_{0}_{1}".format(dfs_node1[i],dfs_node2[j])) for i in set_I for j in set_J}


In [26]:
opt_model = plp.LpProblem(name="MIP Model")
for i in set_I:
    opt_model.addConstraint(
    plp.LpConstraint(e=plp.lpSum(x_vars[i,j] for j in set_J),
                     sense=plp.LpConstraintGE,
                     rhs=1,
                     name="constraintI{0}".format(i)))

for j in set_J:
    opt_model.addConstraint(
    plp.LpConstraint(e=plp.lpSum(x_vars[i,j] for i in set_I),
                     sense=plp.LpConstraintGE,
                     rhs=1,
                     name="constraintJ{0}".format(j))) 
for i in dfs_node1:
    for j in dfs_node2:
        son_i = [dfs_node1[k] for k in range(i.dfs,i.dfs+i.num_son)];
        son_j = [dfs_node2[k] for k in range(j.dfs,j.dfs+j.num_son)];
        for k in i.path:
            for l in j.path:
                if(k==[]or l==[]):
                    continue;
                # print(i,j,k,l)
                opt_model.addConstraint(
                plp.LpConstraint(e=x_vars[i.dfs,j.dfs]+x_vars[i.dfs,l.dfs]+x_vars[k.dfs,j.dfs],
                                sense=plp.LpConstraintLE,
                                rhs=2,
                                name="constraint{}_{}_{}_{}_1".format(i,j,k,l)))
for i in dfs_node1:
    if(len(i.son)==2):
        l=i.son[0];
        r=i.son[1];
        for j in dfs_node2:
            opt_model.addConstraint(
            plp.LpConstraint(e=x_vars[l.dfs,j.dfs]+x_vars[r.dfs,j.dfs]-x_vars[i.dfs,j.dfs],
                                sense=plp.LpConstraintLE,
                                rhs=1,
                                name="constraint{}_{}_{}_{}_1".format(i,j,k,l)))
                

objective = plp.lpSum(x_vars[i,j] * c[i,j] for i in set_I for j in set_J)
opt_model.sense = plp.LpMaximize
opt_model.setObjective(objective)
opt_model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/dongjiajie/opt/anaconda3/lib/python3.8/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/bm/69jvgsdj3rxch558kx9yp9y00000gn/T/c4e6385c4f6144ada8bb7fd58c48d0e8-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/bm/69jvgsdj3rxch558kx9yp9y00000gn/T/c4e6385c4f6144ada8bb7fd58c48d0e8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 352 COLUMNS
At line 1676 RHS
At line 2024 BOUNDS
At line 2105 ENDATA
Problem MODEL has 347 rows, 80 columns and 1147 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 150 - 0.00 seconds
Cgl0004I processed model has 344 rows, 79 columns (79 integer (79 of which binary)) and 1126 elements
Cutoff increment increased from 1e-05 to 9.9999
Cbc0045I 15 integer variables out of 79 objects (79 integer) have cost of -10 - high priority
Cbc00

1

In [27]:
print('SOLUTION:')
for v in opt_model.variables():
    print(f'\t\t{v.name} = {v.varValue}')

print('\n') # Prints a blank line
print(f'OBJECTIVE VALUE: {opt_model.objective.value()}')

SOLUTION:
		x_b1_r1 = 1.0
		x_b1_r10 = 1.0
		x_b1_r2 = 0.0
		x_b1_r3 = 0.0
		x_b1_r4 = 0.0
		x_b1_r5 = 1.0
		x_b1_r6 = 1.0
		x_b1_r7 = 1.0
		x_b1_r8 = 1.0
		x_b1_r9 = 1.0
		x_b2_r1 = 0.0
		x_b2_r10 = 0.0
		x_b2_r2 = 1.0
		x_b2_r3 = 0.0
		x_b2_r4 = 0.0
		x_b2_r5 = 0.0
		x_b2_r6 = 0.0
		x_b2_r7 = 0.0
		x_b2_r8 = 0.0
		x_b2_r9 = 0.0
		x_b3_r1 = 0.0
		x_b3_r10 = 0.0
		x_b3_r2 = 1.0
		x_b3_r3 = 0.0
		x_b3_r4 = 0.0
		x_b3_r5 = 0.0
		x_b3_r6 = 0.0
		x_b3_r7 = 0.0
		x_b3_r8 = 0.0
		x_b3_r9 = 0.0
		x_b4_r1 = 0.0
		x_b4_r10 = 0.0
		x_b4_r2 = 0.0
		x_b4_r3 = 1.0
		x_b4_r4 = 1.0
		x_b4_r5 = 0.0
		x_b4_r6 = 0.0
		x_b4_r7 = 0.0
		x_b4_r8 = 0.0
		x_b4_r9 = 0.0
		x_b5_r1 = 0.0
		x_b5_r10 = 0.0
		x_b5_r2 = 0.0
		x_b5_r3 = 1.0
		x_b5_r4 = 0.0
		x_b5_r5 = 0.0
		x_b5_r6 = 0.0
		x_b5_r7 = 0.0
		x_b5_r8 = 0.0
		x_b5_r9 = 0.0
		x_b6_r1 = 0.0
		x_b6_r10 = 0.0
		x_b6_r2 = 0.0
		x_b6_r3 = 0.0
		x_b6_r4 = 0.0
		x_b6_r5 = 1.0
		x_b6_r6 = 0.0
		x_b6_r7 = 0.0
		x_b6_r8 = 0.0
		x_b6_r9 = 0.0
		x_b7_r1 = 0.0
		x_b7_r